#  Zillow Project: Data Preparation
---

## MVP Objectives
- Remove/Impute missing values.
- Create functions to reproduce prepared dataset
- Move functions to a seperate file named `prepare.py`


In [26]:
# Import libraries to manipulate data structures and visualize numeric data.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from 

In [2]:
# Read in the zillow dataset and assign to a variable
df = pd.read_csv('./data/raw/zillow.csv')

In [3]:
# My first dataset with +2 million rows :)
df.shape

(2985217, 59)

In [4]:
# Initial look at our zillow dataset
df.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2016.0,9.0,NaN,NaN,NaN,NaN
1,1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,2,10843547,NaN,NaN,NaN,0.0,0.0,5.0,NaN,NaN,...,1.0,NaN,660680.0,1434941.0,2016.0,774261.0,20800.37,NaN,NaN,NaN
3,3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,6.0,NaN,...,1.0,NaN,580059.0,1174475.0,2016.0,594416.0,14557.57,NaN,NaN,NaN
4,4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,...,1.0,NaN,196751.0,440101.0,2016.0,243350.0,5725.17,NaN,NaN,NaN


Our dataset has a bunch of missing values! This is only the first 5 rows! Let's see how many values are missing in each column.

In [5]:
# Using `isnull()` and `notnull()` we can calculate the number of missing values and non-null values.
nulls = df.isnull().sum()
non_nulls = df.notnull().sum()

# Add missing values and non-null values together to get the total number values in each column.
total_values = nulls + non_nulls

# Create a variable to store the percentage of missing values in each column.
pct_missing = (nulls/total_values).sort_values(ascending=False)

# Perform formatting to clearly see the percentage of missing values in each column.
pct_missing_chart = pct_missing.apply("{0:.2%}".format)

# Display table to the user showing the percentage of missing values in each column.
print('Percentage of values missing per column')
print('-' * 39)
print(f"{pct_missing_chart}")

Percentage of values missing per column
---------------------------------------
storytypeid                     99.95%
basementsqft                    99.95%
yardbuildingsqft26              99.91%
fireplaceflag                   99.83%
architecturalstyletypeid        99.80%
typeconstructiontypeid          99.77%
finishedsquarefeet13            99.74%
buildingclasstypeid             99.57%
pooltypeid10                    99.43%
decktypeid                      99.42%
finishedsquarefeet6             99.28%
poolsizesum                     99.06%
pooltypeid2                     98.89%
hashottuborspa                  98.32%
taxdelinquencyflag              98.11%
taxdelinquencyyear              98.11%
yardbuildingsqft17              97.27%
finishedsquarefeet15            93.63%
finishedsquarefeet50            93.17%
finishedfloor1squarefeet        93.17%
fireplacecnt                    89.51%
threequarterbathnbr             89.40%
pooltypeid7                     83.05%
poolcnt                

Before we move ahead let's drop columns with more than __9.14%__ of values missing. Imputing values in columns with > 9.14% of values missing is a waste of time. We need to make our dataset _Robust™_.
- If we can impute values in columns: `lotsizesquarefeet`, `finishedsquarefeet12`, great. If not, we can drop them.

In [6]:
columns_to_drop = pct_missing[pct_missing > .3358].index.to_list()

In [7]:
df_subset = df.drop(columns=columns_to_drop)

In [8]:
# My eyes are happy
df_subset.sample(10)

,id,parcelid,bathroomcnt,bedroomcnt,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,latitude,...,regionidcounty,regionidzip,roomcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock
72447,72447,14499041,2.5,4.0,2.5,2431.0,2431.0,6059.0,2.0,33648079.0,...,1286.0,96989.0,0.0,1993.0,291743.0,365872.0,2016.0,74129.0,3716.46,6.059032e+13
934621,934621,13021371,2.0,3.0,2.0,1168.0,1168.0,6037.0,2.0,34144518.0,...,3101.0,96464.0,0.0,1973.0,54722.0,74604.0,2016.0,19882.0,1063.20,6.037401e+13
1468843,1468843,12395153,2.0,3.0,2.0,1430.0,1430.0,6037.0,2.0,33972099.0,...,3101.0,96197.0,0.0,1950.0,97858.0,213199.0,2016.0,115341.0,3030.63,6.037502e+13
1066460,1066460,12489056,1.0,2.0,1.0,1066.0,1066.0,6037.0,1.0,33845297.0,...,3101.0,96242.0,0.0,1950.0,69650.0,251604.0,2016.0,181954.0,3174.15,6.037571e+13
2152372,2152372,11725651,4.0,3.0,4.0,2641.0,2641.0,6037.0,4.0,34031812.0,...,3101.0,95997.0,0.0,1930.0,119801.0,136757.0,2016.0,16956.0,1852.33,6.037220e+13
2687070,2687070,11181218,3.0,3.0,3.0,3047.0,3047.0,6037.0,3.0,34284898.0,...,3101.0,96356.0,0.0,1977.0,279697.0,440248.0,2016.0,160551.0,5557.63,6.037108e+13
1044755,1044755,17232631,3.0,5.0,3.0,2031.0,2031.0,6111.0,3.0,34290741.0,...,2061.0,97116.0,8.0,1968.0,67257.0,84808.0,2016.0,17551.0,1295.86,6.111008e+13
148246,148246,14081618,2.5,3.0,2.5,1673.0,1673.0,6059.0,2.0,33728948.0,...,1286.0,96967.0,6.0,1978.0,188110.0,756000.0,2016.0,567890.0,8398.14,6.059100e+13
526067,526067,11509490,3.0,5.0,3.0,2259.0,NaN,6037.0,3.0,33873203.0,...,3101.0,96109.0,0.0,1960.0,277857.0,782320.0,2016.0,504463.0,8673.31,6.037621e+13
96799,96799,12399580,1.0,2.0,1.0,792.0,792.0,6037.0,1.0,33953839.0,...,3101.0,96100.0,0.0,1950.0,76000.0,380000.0,2016.0,304000.0,4715.38,6.037551e+13


Sweet, our data looks so much better. __We freed up $\approx$ .7+ GB of memory__! Let's press forward.

In [9]:
original_mem_usage = df.memory_usage(index=False).sum()/1_000_000_000
subset_mem_usage = df_subset.memory_usage(index=False).sum()/1_000_000_000

free_memory = original_mem_usage - subset_mem_usage
print(f"{free_memory:.2f}GB of free memory!")

0.79GB of free memory!


### Subset of Zillow data
- Dropped columns missing more than 9.14% of values

In [10]:
# Will return. Retrieving and reproducing data dictionary for these columns.
df_subset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2985217 entries, 0 to 2985216
Data columns (total 26 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   id                            int64  
 1   parcelid                      int64  
 2   bathroomcnt                   float64
 3   bedroomcnt                    float64
 4   calculatedbathnbr             float64
 5   calculatedfinishedsquarefeet  float64
 6   finishedsquarefeet12          float64
 7   fips                          float64
 8   fullbathcnt                   float64
 9   latitude                      float64
 10  longitude                     float64
 11  lotsizesquarefeet             float64
 12  propertycountylandusecode     object 
 13  propertylandusetypeid         float64
 14  rawcensustractandblock        float64
 15  regionidcity                  float64
 16  regionidcounty                float64
 17  regionidzip                   float64
 18  roomcnt               

In [11]:
# Create a function to clean our Zillow dataset
# Note: Create a wrangle function to acquire and prepare the dataset
def prepare_zillow(df):
    '''
    Signature: prepare_zillow(df) -> pandas.core.frame.DataFrame
    Docstring:
    Prepare the zillow dataset for data EDA

    Return DataFrame of zillow dataset

    Parameters
    ----------
    df : pandas.core.frame.DataFrame
    df is the Zillow dataset stored as `zillow.csv`

    Returns
    -------
    DataFrame of the zillow dataset


    Examples
    --------
    To copy the code hold ALT + SHIFT and drag your cursor from the first line to the last.

    >>> df = pd.read_csv('./data/raw/zillow.csv')
    >>>
    >>> nulls = df.isnull().sum()
    >>> non_nulls = df.notnull().sum()
    >>> total_values = nulls + non_nulls
    >>>
    >>> pct_missing = (nulls/total_values).sort_values(ascending=False)
    >>> pct_missing_chart = pct_missing.apply("{0:.2%}".format)
    >>>
    >>> print('Percentage of values missing per column')
    >>> print('-' * 39)
    >>> print(f"{pct_missing_chart}")
    '''
    
    # 1. Drop columns with missing values
    # Calculate the number of missing values and non-null values.
    nulls = df.isnull().sum()
    non_nulls = df.notnull().sum()

    # Get the total number values in each column.
    total_values = nulls + non_nulls

    # Create a variable to store the percentage of missing values in each column.
    # .3358 is a hard coded value from the original analysis.
    # To reproduce use the code in the docstring
    pct_missing = nulls/total_values
    columns_to_drop = pct_missing[pct_missing > .3358].index.to_list()
    
    # drop columns missing more than 33.58% of data.
    df = df.drop(columns=columns_to_drop)
    
    # drop duplicate columns and index column
    duplicate_columns_to_drop = ['calculatedbathnbr', 'finishedsquarefeet12', 'id']
    df = df.drop(columns=duplicate_columns_to_drop)
    
    df = df[df.propertylandusetypeid.isin([261, 262, 273, 279])]
    
    return df

In [12]:
test = prepare_zillow(df)

In [13]:
# Sweet, the function works!
test.sample(10)

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,fullbathcnt,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,...,regionidcounty,regionidzip,roomcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock
736284,11907409,1.0,3.0,1440.0,6037.0,1.0,34073408.0,-118144194.0,6649.0,0100,...,3101.0,96533.0,0.0,1937.0,108631.0,543158.0,2016.0,434527.0,6414.59,6.037482e+13
1178857,17209037,2.5,4.0,2271.0,6111.0,2.0,34216661.0,-118896566.0,7498.0,1111,...,2061.0,96383.0,8.0,1968.0,272470.0,419179.0,2016.0,146709.0,4431.44,6.111006e+13
1600047,11552377,1.0,3.0,1082.0,6037.0,1.0,34015466.0,-118442037.0,13606.0,0100,...,3101.0,96047.0,0.0,1950.0,28407.0,101480.0,2016.0,73073.0,1675.77,6.037271e+13
1608800,14531688,2.0,2.0,1272.0,6059.0,2.0,33808949.0,-117971934.0,NaN,122,...,1286.0,97023.0,6.0,1964.0,65796.0,152291.0,2016.0,86495.0,2107.94,6.059088e+13
1932760,12724551,2.0,4.0,1544.0,6037.0,2.0,33919406.0,-118091633.0,5525.0,0100,...,3101.0,96193.0,0.0,1952.0,83943.0,207855.0,2016.0,123912.0,2836.48,6.037550e+13
366349,12658271,1.0,2.0,713.0,6037.0,1.0,33341653.0,-118324469.0,1019.0,0100,...,3101.0,96207.0,0.0,1912.0,178493.0,498305.0,2016.0,319812.0,7301.18,6.037599e+13
2836386,11215524,3.0,3.0,1742.0,6037.0,3.0,34567902.0,-118020491.0,8246.0,0100,...,3101.0,97330.0,0.0,1991.0,163122.0,203899.0,2016.0,40777.0,3408.84,6.037911e+13
2366193,13005467,3.0,4.0,2160.0,6037.0,3.0,34094547.0,-118042165.0,11414.0,0100,...,3101.0,96517.0,0.0,1947.0,174472.0,257899.0,2016.0,83427.0,3674.79,6.037432e+13
1155921,12453058,1.0,2.0,763.0,6037.0,1.0,33864048.0,-118190781.0,6214.0,0100,...,3101.0,96239.0,0.0,1947.0,69000.0,347000.0,2016.0,278000.0,4172.33,6.037570e+13
839695,11655687,4.0,4.0,3035.0,6037.0,4.0,34112240.0,-118401238.0,18993.0,0101,...,3101.0,96086.0,0.0,1969.0,1286934.0,2619706.0,2016.0,1332772.0,31588.00,6.037261e+13


In [14]:
df = prepare_zillow(df)

In [15]:
df.shape

(2152864, 23)

In [16]:
df.fips.value_counts()

6037.0    1431812
6059.0     555077
6111.0     165975
Name: fips, dtype: int64

In [17]:
# Data dictionary template
numeric_columns = df.select_dtypes(exclude='O')

for column in numeric_columns.columns.to_list():
    print(column, numeric_columns[column].min(), numeric_columns[column].max())

parcelid 10711725 169601949
bathroomcnt 0.0 32.0
bedroomcnt 0.0 25.0
calculatedfinishedsquarefeet 1.0 952576.0
fips 6037.0 6111.0
fullbathcnt 1.0 32.0
latitude 33339912.0 34819650.0
longitude -119475780.0 -117554316.0
lotsizesquarefeet 100.0 371000512.0
propertylandusetypeid 261.0 279.0
rawcensustractandblock 60371011.101 61110091.003010996
regionidcity 3491.0 396556.0
regionidcounty 1286.0 3101.0
regionidzip 95982.0 399675.0
roomcnt 0.0 96.0
yearbuilt 1801.0 2016.0
structuretaxvaluedollarcnt 1.0 66404932.0
taxvaluedollarcnt 1.0 98428909.0
assessmentyear 2000.0 2016.0
landtaxvaluedollarcnt 1.0 88921951.0
taxamount 1.85 1337755.86
censustractandblock -1.0 483030105084015.0


square feet of home, number of bedrooms, and number of bathrooms.

In [23]:
def prepare_zillow_mvp(df):
    '''
    
    '''
    df = df[['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet', 'taxvaluedollarcnt']]
    return df

In [24]:
df = pd.read_csv('./data/raw/zillow.csv')
df = prepare_zillow_mvp(df)

In [25]:
df.head()

,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt
0,0.0,0.0,NaN,9.0
1,0.0,0.0,NaN,27516.0
2,0.0,0.0,73026.0,1434941.0
3,0.0,0.0,5068.0,1174475.0
4,0.0,0.0,1776.0,440101.0
